# Initiazing training files, folders and variables

In [ ]:
import json
import os
with open('../paths.json', 'r') as f:
    paths = json.load(f)
    ptdata = paths["paths"][paths["cloud"]]["training_data"]

In [ ]:
if(not(os.path.isdir(ptdata))):
    !mkdir $ptdata

if(not(os.path.isdir(ptdata+"/TL2"))):
    !mkdir $ptdata/TL2

if(not(os.path.isdir(ptdata+"/TL2/training_checkpoints"))):
    !mkdir $ptdata/TL2/training_checkpoints

if(not(os.path.isdir(ptdata+"/TL2/training_output"))):
    !mkdir $ptdata/TL2/training_output

if(not(os.path.isdir(ptdata+"/TL2/training_errors"))):
    !mkdir $ptdata/TL2/training_errors

if(not(os.path.isdir(ptdata+"/TL2/model_weights"))):
    !mkdir $ptdata/TL2/model_weights

if(not(os.path.isfile(ptdata+"/TL2/last_epoch.num"))):
    epf = open(ptdata+"/TL2/last_epoch.num", "w")
    epf.write("0")
    epf.close()

if(not(os.path.isfile(ptdata+"/TL2/model_weights/model-ep0.hdf5"))):
    import numpy as np
    print('numpy version : ', np.version.version)
    import tensorflow as tf
    print('tensorflow version : ', tf.__version__)
    import tensorflow.keras
    print('tensorflow keras version : ', tensorflow.keras.__version__)
    from tensorflow.keras.models import Sequential
    from tensorflow.keras.layers import Activation
    vgg16_model = tensorflow.keras.applications.vgg16.VGG16()
    #type(vgg16_model)
    #vgg16_model.summary()
    model = Sequential()
    for layer in vgg16_model.layers:
        model.add(layer)
    model.pop()
    model.add(tensorflow.keras.layers.Dense(10, activation='softmax'))
    for layer in model.layers:
        layer.trainable = True
    #model.summary()
    model.save(ptdata+"/TL2/model_weights/model-ep0.hdf5", include_optimizer=False)
    if(not(os.path.isfile(ptdata+"/TL2/model_weights/model-ep0.hdf5"))):
        print("Could not save model !!!")
    else:
        print("model saved")

!ls $ptdata/TL2
print("==> last_epoch.num :")
!cat $ptdata/TL2/last_epoch.num
print()
print("==> model_weights :")
!ls $ptdata/TL2/model_weights
print("==> training_checkpoints :")
!ls $ptdata/TL2/training_checkpoints
print("==> training_output :")
!ls $ptdata/TL2/training_output

# Start training

In [ ]:
# uncomment next line if this is the training environment
#!echo 1 > ../prod.flag

In [ ]:
total_epochs = 200
epf = open(ptdata+"/TL2/last_epoch.num", "r")
last_epoch = int(epf.readlines()[0])
epf.close()
while(True):
    if(last_epoch == total_epochs):
        break
    print("Last epoch number : ", last_epoch)
    next_epoch = last_epoch + 1
    print("Next epoch number : ", next_epoch)
    !jupyter nbconvert --to script ./Training.ipynb
    training_output = ptdata+"/TL2/training_output/training" + str(next_epoch) + ".out"
    !ipython ./Training.py > $training_output
    !rm Training.py
    f = open(training_output, 'r')
    lines = f.read().splitlines()
    last_line = lines[-1]
    if(last_line == "model saved"):
        epf = open(ptdata+"/TL2/last_epoch.num", "w")
        epf.write(str(next_epoch))
        epf.close()
        !cat $ptdata/TL2/last_epoch.num
        last_epoch = next_epoch
    else:
        training_output_e = ptdata + '/TL2/training_errors/training' + str(next_epoch) + ".out"
        !mv $training_output $training_output_e
    print()
    print("*********************************************************************************")
    print()
print("Finished !")